<a href="https://colab.research.google.com/github/ishankagrawal/LDA-model-for-Cardiotoxicity-prediction/blob/main/LDA_model_for_screening_pharmaceutical_compounds_with_hERG_inhibitory_activity_(cardiotoxicity).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2020-11-30 13:41:57--  https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh [following]
--2020-11-30 13:41:57--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | / - \ | / - \ | / - \ | / - 

In [ ]:
pip install -y boost-cpp boost py-boost


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y


In [ ]:
import numpy as np
import pandas as pd
import keras as k
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization
from keras.layers.core import Dense
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from rdkit import Chem, DataStructs
from rdkit.Chem import rdMolDescriptors, Draw
from rdkit.Chem.Draw import IPythonConsole
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('/content/drive/My Drive/Drug Discovery Hackathon/data1.xlsx')

In [ ]:
data = df.to_numpy()
print(data)

[[1 '[11CH3]Oc1ccc2cccc(N3CCN(CCCCN4N=CC(=O)N(C)C4=O)CC3)c2c1' 'Blocker']
 [2
  '[2H]C([2H])([2H])Oc1cc(ncc1C#N)C(O)CN2CCN(C[C@H](O)c3ccc4C(=O)OCc4c3C)CC2'
  'Blocker']
 [3
  '[2H]C([2H])(O)CN1CCN(CC1)c2cnc3cc(cc(NCc4cccc(c4)[N+](=O)[O-])c3c2)C(F)(F)F'
  'Blocker']
 ...
 [9202 'C(=Nc1cc2c(N(CC[C@H]3N(CCC3)C)CCC2)cc1)(c4sccc4)N' 'BLINDED']
 [9203 'C(=O)(c1c(NC(=O)c2c3c(c(cc2)CS(=O)C)cccc3)cccn1)NCC4CCOCC4'
  'BLINDED']
 [9204
  'O=C1[C@H]2N(c3ccc(OCCCCCNCC(=O)Nc4c(Cl)cc(cc4Cl)CN=C(N1)N)cc3)CCC2'
  'BLINDED']]


In [ ]:
mols = [None]*data.shape[0]
for i in range(0,data.shape[0]):
  mols[i] = Chem.rdmolfiles.MolFromSmiles(data[i,1])

1ccc(O)c7c1[C@@]689
1ccc(O)c7c1[C@@]689'
)Cc5ccccc5)CC(=O)N
)Cc5ccccc5)CC(=O)N'
C)CC(C)C
C)CC(C)C'
C)C
C)C'
@H]7O)[C@H](O)C[C@]6(C)[C@H]5CC[C@]4(C)[C@H]3[C@@H]2C
@H]7O)[C@H](O)C[C@]6(C)[C@H]5CC[C@]4(C)[C@H]3[C@@H]2C'
)C[C@H](C)N1
)C[C@H](C)N1'
C@H](C)N1
C@H](C)N1'
c6c78)[nH]c7cccc8)CC9CC(O)(CC)CN(C9)CC6
c6c78)[nH]c7cccc8)CC9CC(O)(CC)CN(C9)CC6'
])(=O)=O)cc(S([O-])(=O)=O)cc6
])(=O)=O)cc(S([O-])(=O)=O)cc6'
@H]([C@H]45)CCCC5
@H]([C@H]45)CCCC5'
c7cccc8[nH]cc6c78
c7cccc8[nH]cc6c78'
O[C@H](C)C[C@@H]([C@H]3O)N(C)C(C)C)[C@](C)(O)C[C@@H](C)[C@H](O)[C@H](C)[C@@H](O)[C@]1(C)O
O[C@H](C)C[C@@H]([C@H]3O)N(C)C(C)C)[C@](C)(O)C[C@@H](C)[C@H](O)[C@H](C)[C@@H](O)[C@]1(C)O'
O[C@H](C)C[C@@H]([C@H]3O)N(C)C(C)C)[C@](C)(O)C[C@@H](C)[C@H](OCC(=O)NC)[C@H](C)[C@@H](O)[C@]1(C)O
O[C@H](C)C[C@@H]([C@H]3O)N(C)C(C)C)[C@](C)(O)C[C@@H](C)[C@H](OCC(=O)NC)[C@H](C)[C@@H](O)[C@]1(C)O'
O[C@H](C)C[C@@H]([C@H]3O)N(C)C)[C@@](C)(C[C@@H](C)C(=O)[C@H](C)[C@@H](O)[C@]1(C)O)OC
O[C@H](C)C[C@@H]([C@H]3O)N(C)C)[C@@](C)(C[C@@H](C)C(=O)[

In [ ]:
#fps=[None]*data.shape[0]
fps=[]
bi={}
errors = []
for i in range(0,data.shape[0]):
  try:
    fps.append(rdMolDescriptors.GetMorganFingerprintAsBitVect(mols[i], radius=2, bitInfo= bi, nBits=256))
  except:
    errors.append(i)
    pass

In [ ]:
errors

[7,
 276,
 307,
 432,
 583,
 706,
 707,
 964,
 1089,
 1143,
 1197,
 2210,
 2211,
 2212,
 2213,
 2214,
 2215,
 2216,
 2217,
 2218,
 2219,
 2220,
 2221,
 2240,
 2241,
 2242,
 2243,
 2244,
 2245,
 2246,
 2253,
 3406,
 3492,
 3511,
 3679,
 3948,
 5291,
 5292,
 5394,
 5395,
 5396,
 5397,
 5398,
 6521,
 6537,
 6538,
 6539,
 6540,
 6711,
 7454,
 8606,
 8607,
 8608]

In [ ]:
for i in range(0,len(errors)):
  del mols[errors[i]]
print(len(mols))

9151


In [ ]:
data = np.delete(data, errors, 0)

In [ ]:
data.shape

(9151, 3)

In [ ]:
np_fps_array = []
for fp in fps:
  arr = np.zeros((1,), dtype= int)
  DataStructs.ConvertToNumpyArray(fp, arr)
  np_fps_array.append(arr)

In [ ]:
data.shape

(9151, 3)

In [ ]:
y = np.zeros((data.shape[0],2))

In [ ]:
for i in range(0,data.shape[0]):
  if data[i,2]=='Blocker':
    y[i,0]=1
  if data[i,2]=='Non-Blocker':
    y[i,1]=1

In [ ]:
for i in range(0,data.shape[0]):
  if data[i,2]=='BLINDED':
    print(i)
    break
#Result: i=9053

9054


In [ ]:
y_blind = y[9053:,:]
y=y[0:9053,:]
y_blind.shape

(98, 2)

In [ ]:
x_blind = np.array(np_fps_array)[9053:,:]
x=np.array(np_fps_array)[0:9053,:]



In [ ]:
y_uni = y.shape[0]
blocked = []
for i in range(0,y_uni):
  if y[i,0]==1:
    blocked.append(i)

In [ ]:
len(blocked)

7517

In [ ]:
y_uni

9053

In [ ]:
toremove = blocked[1535:]

In [ ]:
yuni = np.delete(y, toremove, 0)
xuni = np.delete(x, toremove, 0)

In [ ]:
xy =np.append(x,y,axis=1)

In [ ]:
np.random.shuffle(xy)

In [ ]:
xy.shape
print(xy)

[[0. 0. 0. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 ...
 [0. 1. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 1. ... 0. 1. 0.]]


In [ ]:
def split(xy,ratio):
  s = int(ratio*x.shape[0])
  x_train = xy[0:s,:256]
  y_train = xy[0:s,256:]
  x_test = xy[s:,:256]
  y_test = xy[s:,256:]
  return (x_train,y_train,x_test,y_test)

In [ ]:
def get_covs(X_train,Y_train):
  pos = []
  neg = []
  n,d = X_train.shape
  for i in range(n):
    if(Y_train[i][0]==0):
      neg.append(i)
    else:
      pos.append(i)
  m1 = X_train[pos]
  m0 = X_train[neg]
  mew1 = np.mean(m1,axis=0)
  mew0 = np.mean(m0,axis=0)
  m1 = np.transpose(m1)
  m0 = np.transpose(m0)
  covt = np.cov(np.transpose(X_train))
  cov1 = np.cov(m1)
  cov0 = np.cov(m0)
  return (covt,cov0,cov1,mew0,mew1)

In [ ]:
x_train,y_train,x_test,y_test = split(xy,0.8)
y_train = y_train[:,[0]]
y_test = y_test[:,[0]] 
print(x_train)
todelete = []
for i in range(x_test.shape[0]):
  
  if(y_test[i][0]==1):
    todelete.append(i)
print(x_train.shape)
k = int(len(todelete)*0.75)
x_t = np.delete(x_test,todelete[:k],0)
y_t = np.delete(y_test,todelete[:k],0)
print(x_t.shape)
todelete1 = []
for i in range(x_train.shape[0]):
  
  if(y_train[i][0]==1):
    todelete1.append(i)
k1  = int(len(todelete1)*0.75)
print(x_train.shape)
k = int(len(todelete1)*0.75)
x_t1 = np.delete(x_train,todelete1[:k],0)
y_t1 = np.delete(y_train,todelete1[:k],0)
print(x_t1.shape)


[[0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 1.]]
(7242, 256)
(673, 256)
(7242, 256)
(2743, 256)


In [ ]:
covt,cov0,cov1,mew0,mew1 = get_covs(x_train,y_train)
cov1.shape

(256, 256)

In [ ]:
def qda(cov0,cov1,mew0,mew1,X_test,Y_test,t):
  n,d = X_test.shape
  res = np.zeros((n,1))
  s = 0
  cov0 = np.linalg.inv(cov0)
  cov1 = np.linalg.inv(cov1)
  for i in range(n):
    t1 = np.log(np.linalg.det(cov1))
    t2 = np.matmul(X_test[i]-mew1,np.matmul(cov1,np.transpose(X_test[i]-mew1)))
    t3 = np.log(np.linalg.det(cov0))
    t4 = np.matmul(X_test[i]-mew0,np.matmul(cov0,np.transpose(X_test[i]-mew0)))
    if(t3+t4>t1+t2+np.log(t)):
      res[i][0] = 1
    if(Y_test[i][0]==res[i][0]):
      s+=1
  print("accuracy :" + str(100*(s/n)))
  return res

In [ ]:
pred = qda(cov0,cov1,mew0,mew1,x_test,y_test,1)

/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py:2116: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


accuracy :16.178906681391496


In [ ]:
def lda(cov,mew0,mew1,X_test,Y_test,th):
  n,d = X_test.shape
  res = np.zeros((n,1))
  s = 0
  
  mew0 = np.transpose(mew0)
  mew1 = np.transpose(mew1)
  w = np.matmul(np.linalg.inv(cov),mew1-mew0)
  c = 0.5*np.dot(w,mew1+mew0)

  for i in range(n):
    
    if(np.dot(X_test[i],w)>c-np.log(th)):
      res[i][0] = 1
    if(Y_test[i][0]==res[i][0]):
      s+=1
  print("accuracy :" + str(100*(s/n)))
  return res

In [ ]:
pred = None
th = 1
while(th<11):
  print("for threshhold" + "= "+ str(th) + ":") 
  pred = lda(cov1,mew0,mew1,x_test,y_test,th)
  th+=1

for threshhold= 1:
accuracy :78.96189950303699
for threshhold= 2:
accuracy :82.16454997239094
for threshhold= 3:
accuracy :83.37934842628381
for threshhold= 4:
accuracy :83.8210933186085
for threshhold= 5:
accuracy :84.64936499171728
for threshhold= 6:
accuracy :84.81501932633904
for threshhold= 7:
accuracy :85.14632799558255
for threshhold= 8:
accuracy :85.09110988404197
for threshhold= 9:
accuracy :85.20154610712314
for threshhold= 10:
accuracy :85.42241855328547


In [ ]:
def wilks(X,Y):
  pos = []
  neg = []
  n,d = X.shape
  for i in range(n):
    if(Y[i][0]==0):
      neg.append(i)
    else:
      pos.append(i)
  m1 = X[pos]
  m0 = X[neg]
  
  
  mew1 = np.mean(m1,axis=0)
  mew0 = np.mean(m0,axis=0)
  
  mew = np.mean(X,axis=0)
  
  mew0 = mew0.reshape(1,256)
  mew1 = mew1.reshape(1,256)
  mew = mew.reshape(1,256)
 
  n0,d = m0.shape
  n1,d = m1.shape
  
  H = np.zeros((d,d))
  H = H + n1*np.matmul(np.transpose(mew1-mew),mew1-mew)
  H = H + n0*np.matmul(np.transpose(mew0-mew),mew0-mew)
  
 
  E = np.zeros((d,d))
  for v in m1:
    E+=np.matmul(np.transpose(v-mew1),v-mew1)
  for v in m0:
    E+= np.matmul(np.transpose(v-mew0),v-mew0)
  detA = np.linalg.det(E)
  
  detB = np.linalg.det(E+H)
  
  return abs(detA)/abs(detB)

In [ ]:
print(wilks(x_test,pred))

nan


/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py:2116: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
def get_result(cov,mew0,mew1,X_test,th):
  n,d = X_test.shape
  res = np.zeros((n,1))
  s = 0
  
  mew0 = np.transpose(mew0)
  mew1 = np.transpose(mew1)
  w = np.matmul(np.linalg.inv(cov),mew1-mew0)
  c = 0.5*np.dot(w,mew1+mew0)

  for i in range(n):
    
    if(np.dot(X_test[i],w)>c-np.log(th)):
      res[i][0] = 1

  return res

In [ ]:
y_pred = get_result(cov1,mew0,mew1,x_blind,10)
iter = 1
f = open('/content/drive/My Drive/Drug Discovery Hackathon/' + 'iupac.txt','w')
for i in range(98):
  smiles = data[i+9053][1]
  print(smiles)
  compounds = pubchempy.get_compounds(smiles, namespace='smiles')
  match = compounds[0]
  try:
    if(y_pred[i][0]==1):
      f.write(match.iupac_name + ' Blocker' + '\n')
    else:
      f.write(match.iupac_name + ' Non-Blocker'+ '\n')
  except:
    continue
  iter+=1
f.close()




ONC(=O)\C=C\c1ccc2CN(Cc2c1)S(=O)(=O)c3ccc(cc3)C(F)(F)F
ONC(=O)\C=C\c1ccc2CN(Cc2c1)S(=O)(=O)c3cccc(c3)C(F)(F)F
ONC(=O)\C=C\c1ccc2CN(Cc2c1)S(=O)(=O)c3ccccc3
ONC(=O)\C=C\c1ccc2CN(Cc2c1)S(=O)(=O)c3cccnc3
ONC(=O)\C=C\c1ccc2CN(Cc2c1)S(=O)(=O)c3cccs3
ONC(=O)\C=C\c1ccc2CN(Cc2c1)S(=O)(=O)c3ccncc3
ONC(=O)\C=C\c1ccc2CN(Cc2c1)S(=O)(=O)Cc3ccccc3
ONC(=O)\C=C\c1ccc2CN(Cc3ccccc3)Cc2c1
ONC(=O)\C=C\c1ccc2CN(CCc3c[nH]c4ccccc34)Cc2c1
ONC(=O)\C=C\c1ccc2CN(CCC3CCCCC3)Cc2c1
ONC(=O)\C=C\c1ccc2CN(CCc3cnn4ccccc34)Cc2c1
ONC(=O)\C=C\c1ccc2OC3(CCN(Cc4ccc(F)cc4)CC3)CC(=O)c2c1
ONC(=O)\C=C\c1ccc2OC3(CCN(Cc4ccccc4)CC3)CC(=O)c2c1
ONC(=O)\C=C\c1cccc(c1)S(=O)(=O)Nc2ccccc2
ONC(=O)C1(CCC2(CCNCC2)C1)S(=O)(=O)c3ccc(Oc4ccc(cc4)C(=O)N5CCCC5)cc3
ONC(=O)C1(CCC2(CCNCC2)C1)S(=O)(=O)c3ccc(Oc4ccc(OC(F)(F)F)cc4)cc3
ONC(=O)C1(CCC2(CCNCC2)C1)S(=O)(=O)c3ccc(OCc4ccc(OC(F)(F)F)cc4)cc3
ONC(=O)c1cnc(nc1)N2C[C@H]3CN(Cc4ccc5ccccc5c4)C[C@H]3C2
ONC(=O)CCCCCCC(=O)Nc1ccc2ccnc(Nc3ccc(Br)cc3F)c2c1
ONC(=O)CCCCCCC(=O)Nc1ccc2ccnc(Nc3ccc4ccccc4c3)c2c1


In [ ]:
f = open('/content/drive/My Drive/Drug Discovery Hackathon/' + 'iupac.txt','r+')
f.seek(0)
for line in f:
  if(line[len(line)-7:]=='Blocker'):
    f.write(line[:len(line)-7]+(' '*(100-len(line)-7))+'Blocker \n')

In [ ]:
import requests


CACTUS = "https://cactus.nci.nih.gov/chemical/structure/{0}/{1}"


def smiles_to_iupac(smiles):
    rep = "iupac_name"
    url = CACTUS.format(smiles, rep)
    response = requests.get(url)
    response.raise_for_status()
    return response.text

In [ ]:
import pubchempy


# Use the SMILES you provided
smiles = 'CCCCCCCCCCCCCC'
compounds = pubchempy.get_compounds(smiles, namespace='smiles')
match = compounds[0]
print(match.iupac_name)

tetradecane


In [ ]:
!pip install pubchempy

  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13828 sha256=a7e8505547c8c19c6e1f7655520a549003e04b7a221262da2295f910d15d7f21
  Stored in directory: /root/.cache/pip/wheels/7c/3d/8c/8192697412e9899dc55bbbb08bbc1197bef333caaa2a71c448
Successfully built pubchempy
